# US City Isochrones using Google Maps API and CARTO Isolines

This note book will do the following:

1. Take a list of the largest US Cities by Population (pulled from here:
https://gist.github.com/Miserlou/11500b2345d3fe850c92)


2. Find the Lat/Lon Coordinates of each "Downtown" using the Google Maps Geocode API: https://developers.google.com/maps/documentation/geocoding/start


3. Calculate what a 60 Minute Isochrone from the downtown Pin Location using CARTO Isolines:
https://carto.com/help/glossary/#isolines


4. Display those Isochrone shapes in a Kepler Map (My preferred Mapping Tool) 

In [1]:
#import python libraries

import pandas as pd
import shapely
import geopandas as gpd
import numpy as np
import geog
from shapely import geometry
from shapely.geometry import Polygon, shape
import googlemaps
import os
import keplergl
import json
from datetime import date

In [2]:
# import carto libraries
from cartoframes.data.services import Isolines
from cartoframes.auth import Credentials, set_default_credentials

# import personal account credentials
set_default_credentials('creds.json')

In [3]:
# initialize google maps connection
# this will be utilized to geocode lat and lon coordinates 
gmaps = googlemaps.Client(key=os.environ.get("GOOGLE_MAPS_API_KEY", ""))

In [4]:
# call the cartogropher function Isolines
iso_service = Isolines()

## 1. Generate List of Cities

Create a list of top 20 US Cities by population and create corresponding data frame to store future information

In [5]:
largest_us_cities_list = ['Downtown New York City, NY',
                          'Downtown Los Angeles, CA',
                          'Downtown Chicago, IL',
                          'Downtown Houston, TX',
                          'Downtown Phoenix, AZ',
                          'Downtown Philadelphia, PA',
                          'Downtown San Antonio, TX',
                          'Downtown San Dieog, CA',
                          'Downtown Dallas, TX',
                          'Downtown Austin, TX',
                          'Downtown San Jose, CA',
                          'Downtown Fort Worth, TX',
                          'Downtown Jacksonville, FL',
                          'Downtown Columbus, OH',
                          'Downtown Charlotte, NC',
                          'Downtown Indianapolis, IN',
                          'Downtown San Francisco, CA',
                          'Downtown Seattle, WA',
                          'Downtown Denver, CO',
                          'Downtown Washinton, DC',
                         ]

In [6]:
# create data frame to store location data
largest_us_cities_df = pd.DataFrame()

largest_us_cities_df['city'] = largest_us_cities_list

#initialize columns as floats
largest_us_cities_df['lat'] = 0.0000000000000000000
largest_us_cities_df['lon'] = 0.0000000000000000000

In [7]:
# check out the dataset
largest_us_cities_df

,city,lat,lon
0,"Downtown New York City, NY",0.0,0.0
1,"Downtown Los Angeles, CA",0.0,0.0
2,"Downtown Chicago, IL",0.0,0.0
3,"Downtown Houston, TX",0.0,0.0
4,"Downtown Phoenix, AZ",0.0,0.0
5,"Downtown Philadelphia, PA",0.0,0.0
6,"Downtown San Antonio, TX",0.0,0.0
7,"Downtown San Dieog, CA",0.0,0.0
8,"Downtown Dallas, TX",0.0,0.0
9,"Downtown Austin, TX",0.0,0.0


## 2. Find the Lat/Lon Coordinates of each City using the Google Maps Geocode API

This will use the google maps "geocode" functionality to pull information for each city, including the pin location of each city. 

Note to do this you will need to generate your own personal API Key. 

In [8]:
# loop through list of cities and find geo coordinates using Google Maps
for row in range(0,len(largest_us_cities_list)):
    geocode_result = gmaps.geocode(largest_us_cities_list[row])
    largest_us_cities_df['lat'][row] = [
                    (
                        result["geometry"]["location"]["lat"]
                    )
                    for result in geocode_result
                ].pop()
    largest_us_cities_df['lon'][row] = [
                    (
                        result["geometry"]["location"]["lng"]
                    )
                    for result in geocode_result
                ].pop()  

/Users/cody/.local/share/virtualenvs/isochrone-d5Mthehx-python/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/cody/.local/share/virtualenvs/isochrone-d5Mthehx-python/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# examine what a typical API Call looks like
geocode_result

[{'address_components': [{'long_name': 'Downtown',
    'short_name': 'Downtown',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Fort Lesley J. McNair',
    'short_name': 'Fort Lesley J. McNair',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Washington',
    'short_name': 'Washington',
    'types': ['locality', 'political']},
   {'long_name': 'District of Columbia',
    'short_name': 'DC',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']}],
  'formatted_address': 'Downtown, Washington, DC, USA',
  'geometry': {'bounds': {'northeast': {'lat': 38.9096366, 'lng': -77.0239716},
    'southwest': {'lat': 38.9001847, 'lng': -77.0496082}},
   'location': {'lat': 38.9037406, 'lng': -77.0362967},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 38.9096366, 'lng': -77.0239716},
    'southwest': {'lat': 38.9001847, '

In [10]:
# examine what the largest cities dataframe looks like:
largest_us_cities_df.head()

,city,lat,lon
0,"Downtown New York City, NY",40.720860,-74.000669
1,"Downtown Los Angeles, CA",34.040713,-118.246769
2,"Downtown Chicago, IL",41.875672,-87.624347
3,"Downtown Houston, TX",29.755970,-95.357319
4,"Downtown Phoenix, AZ",33.451717,-112.074464


## 3. Calculate what a 60 Minute Isochrone from the downtown Pin Location using CARTO Isolines

Isochrones are a way of measuring how much distance you can travel in all directions from a specific point given a fixed amount of time. This is a simple way of measuring how much traffic and roadway networks are availabile for each city. 

In [11]:
# create a geo dataframe, this will create a "geometry" column for each city

geo_df = gpd.GeoDataFrame(largest_us_cities_df, geometry=gpd.points_from_xy(largest_us_cities_df.lon, largest_us_cities_df.lat))


In [12]:
geo_df.head()

,city,lat,lon,geometry
0,"Downtown New York City, NY",40.720860,-74.000669,POINT (-74.00067 40.72086)
1,"Downtown Los Angeles, CA",34.040713,-118.246769,POINT (-118.24677 34.04071)
2,"Downtown Chicago, IL",41.875672,-87.624347,POINT (-87.62435 41.87567)
3,"Downtown Houston, TX",29.755970,-95.357319,POINT (-95.35732 29.75597)
4,"Downtown Phoenix, AZ",33.451717,-112.074464,POINT (-112.07446 33.45172)


In [13]:
# pull isochrone data for car transportation for 60 minutes (3600 seconds)
isochrones_gdf, isochrones_metadata = iso_service.isochrones(geo_df, [3600], mode='car')

Success! Isolines created correctly


In [14]:
# calculate the area of each isochrone by converting to the correct map projection and converting from SQ M to SQ KM

isochrones_gdf['city'] = largest_us_cities_df['city']
isochrones_gdf['pin_location'] = geo_df['geometry']
isochrones_gdf['geometry'] = isochrones_gdf.the_geom

isochrones_gdf['the_geom_crs'] = isochrones_gdf.the_geom.to_crs({'proj':'cea'}) 
isochrones_gdf['area_sq_km'] = isochrones_gdf.the_geom_crs.area / 10**6

# rank the cities from largest to smallest in terms of shape size
isochrones_gdf['area_rank'] = isochrones_gdf['area_sq_km'].rank(ascending=False)

In [15]:
isochrones_gdf.head()

,source_id,data_range,the_geom,city,pin_location,geometry,the_geom_crs,area_sq_km,area_rank
0,0,3600,"MULTIPOLYGON (((-74.89243 40.64941, -74.86084 ...","Downtown New York City, NY",POINT (-74.00067 40.72086),"MULTIPOLYGON (((-74.89243 40.64941, -74.86084 ...","MULTIPOLYGON (((-8336986.671 4134923.896, -833...",5247.333500,18.0
1,1,3600,"MULTIPOLYGON (((-118.92563 34.18945, -118.9187...","Downtown Los Angeles, CA",POINT (-118.24677 34.04071),"MULTIPOLYGON (((-118.92563 34.18945, -118.9187...","MULTIPOLYGON (((-13238740.429 3565107.098, -13...",6755.909881,13.0
2,2,3600,"MULTIPOLYGON (((-88.52646 41.86890, -88.51959 ...","Downtown Chicago, IL",POINT (-87.62435 41.87567),"MULTIPOLYGON (((-88.52646 41.86890, -88.51959 ...","MULTIPOLYGON (((-9854720.304 4236881.164, -985...",5461.113932,17.0
3,3,3600,"MULTIPOLYGON (((-96.23749 29.77295, -96.21826 ...","Downtown Houston, TX",POINT (-95.35732 29.75597),"MULTIPOLYGON (((-96.23749 29.77295, -96.21826 ...","MULTIPOLYGON (((-10713108.137 3149419.545, -10...",12817.067467,4.0
4,4,3600,"MULTIPOLYGON (((-113.11249 33.55225, -113.0987...","Downtown Phoenix, AZ",POINT (-112.07446 33.45172),"MULTIPOLYGON (((-113.11249 33.55225, -113.0987...","MULTIPOLYGON (((-12591624.544 3506361.778, -12...",9308.110165,10.0


In [16]:
# save the file to .csv to avoid future API calls
isochrones_gdf.to_csv('isochrones_gdf.csv')

## 4. Visualize the Data in Kepler

Note that Kepler is a personal choice, CARTO, Google, and many other ways to display data on maps :). 

In [17]:
geo_df_kepler = gpd.GeoDataFrame(isochrones_gdf, geometry=isochrones_gdf.the_geom)

In [18]:
geo_df_kepler.drop(['the_geom_crs','the_geom','pin_location'],axis=1,inplace=True)

In [19]:
import kepler_config

In [20]:
# Map the different shapes in Kepler
kepler_map = keplergl.KeplerGl(height=800)
kepler_map.add_data(data = geo_df_kepler, name = 'US City Isochrones')
kepler_map.config = kepler_config.kepler_config

kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'odjlmao', 'type': …

Scrolling through the map you can clearly see that cities in Texas have much larger distances in which you can travel in 60 minutes. Cities near large bodies of water like Chicago, Seattle, and San Diego natrually have smaller shapes because of their location (cannot drive in all directions).

In [21]:
# save kepler config file to save visualization settings
kepler_config = kepler_map.config

In [22]:
#save map
kepler_map.save_to_html(file_name='US_City_Isochrones.html')

Map saved to US_City_Isochrones.html!
